# 0. Install and Import Packages

Read more about the MediaPipe classes to be used at 
- https://google.github.io/mediapipe/solutions/face_mesh.html
- https://google.github.io/mediapipe/solutions/hands.html
- https://google.github.io/mediapipe/solutions/pose.html

In [2]:
 !pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp # MediaPipe is by Google
import cv2 # OpenCV is by Intel

import csv # To export train data i.e. key points detected to CSV file
import os
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score
import pickle 

In [3]:
face = pd.read_csv("coords.csv")

In [4]:
# Drawing Helpers - to draw the keypoints and lines on video feed
# https://github.com/google/mediapipe/blob/master/mediapipe/python/solutions/drawing_utils.py
mp_drawing = mp.solutions.drawing_utils 

# Holistic pipeline integrates separate models for pose, face and hand components
# Each model is optimised for their particular domain
# Read more at https://google.github.io/mediapipe/solutions/holistic.html
mp_holistic = mp.solutions.holistic

# 1. Try Making Detections

In [27]:
# To launch WebCam
cap = cv2.VideoCapture(0)

# Initiate holistic model - https://google.github.io/mediapipe/solutions/holistic.html
# Minimum confidence value ([0.0, 1.0]) from the person-detection model for the detection to be considered successful. Default to 0.5.
# Minimum confidence value ([0.0, 1.0]) from the landmark-tracking model for the pose landmarks to be considered tracked successfully. Default to 0.5.
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read() # Read Feed
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
        image.flags.writeable = False # Image is no longer writeable     
        
        # Make Some Detections
        results = holistic.process(image) # Make prediction
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True # Image is now writeable
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION RGB 2 BGR
        
        # 1. Draw face landmarks and face connections
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Display the Webcam Capture window with window title and keypoints and connections drawn on the feed
        cv2.imshow('Raw Webcam Feed', image)

        # Press q key to terminate webcam capture mode
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Once the q key is clicked, close the capture mode and webcam windows
cap.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV
Some of the Landmarks (Key Points) detected by MediaPipe Holistic Model

### Hand Landmarks
<img src="https://i.imgur.com/8bForKY.png">

### Pose Landmarks
<img src="https://i.imgur.com/AzKNp7A.png">

In [28]:
# Calculate the total number of landmarks detected by Pose model and Face model
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [30]:
# Prepare the list of column names starting with prediction class, coordinates of 1st landmark, coordinates of 2nd landmark, etc
landmarks = ['class']

for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [31]:
# refers to column names in csv
# class refers to the body language to be detected e.g. happy, sad, etc.
# rest of the columns are representing the coordinates
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [32]:
# Write the column names to the csv file
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [73]:
# Set the class name for the specific body language
# [Impt] Change the class name when creating data for a new class
class_name = "Happy"

In [74]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)
        
        # Press q key to terminate webcam capture mode
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [75]:
# Read coordinates csv file
df = pd.read_csv('coords.csv')

In [76]:
# Display first 5 rows
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Sad,0.524292,0.810037,-1.279028,0.989107,0.565088,0.741143,-1.179697,0.988755,0.589510,...,-0.003031,0.0,0.615876,0.721075,0.021968,0.0,0.622517,0.713997,0.022469,0.0
1,Sad,0.523562,0.810187,-1.380470,0.990079,0.561935,0.740872,-1.278919,0.989783,0.584893,...,-0.003120,0.0,0.614124,0.720224,0.020959,0.0,0.620538,0.713706,0.021380,0.0
2,Sad,0.521723,0.809525,-1.400822,0.990983,0.558585,0.739532,-1.293301,0.990727,0.580836,...,-0.001872,0.0,0.610909,0.721197,0.022571,0.0,0.617341,0.714742,0.022983,0.0
3,Sad,0.520213,0.809108,-1.388353,0.991818,0.556583,0.738727,-1.276241,0.991590,0.578610,...,-0.002967,0.0,0.609726,0.720498,0.021407,0.0,0.616134,0.713853,0.021858,0.0
4,Sad,0.519008,0.808903,-1.382275,0.992485,0.555177,0.738464,-1.242853,0.992276,0.577124,...,-0.002414,0.0,0.605670,0.720482,0.021442,0.0,0.612132,0.713920,0.021803,0.0


In [77]:
# Display last 5 rows 
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
279,Happy,0.579028,0.800905,-1.226265,0.996660,0.607931,0.736454,-1.127062,0.997048,0.628344,...,-0.006276,0.0,0.651383,0.712594,0.022473,0.0,0.658304,0.704440,0.023698,0.0
280,Happy,0.576286,0.800849,-1.220473,0.996702,0.605653,0.736373,-1.121444,0.997083,0.626007,...,-0.006545,0.0,0.643902,0.710608,0.021425,0.0,0.650929,0.702492,0.022598,0.0
281,Happy,0.573127,0.800818,-1.211762,0.996770,0.603498,0.736329,-1.113325,0.997132,0.623892,...,-0.006028,0.0,0.643860,0.710730,0.022288,0.0,0.650898,0.702378,0.023495,0.0
282,Happy,0.572295,0.800582,-1.217214,0.996809,0.603029,0.736206,-1.119309,0.997154,0.623281,...,-0.006218,0.0,0.641802,0.710640,0.022382,0.0,0.648791,0.702202,0.023608,0.0
283,Happy,0.566041,0.800434,-1.191320,0.996842,0.596209,0.736121,-1.091291,0.997173,0.616591,...,-0.006759,0.0,0.636814,0.708714,0.021013,0.0,0.643707,0.700722,0.022086,0.0


In [78]:
# View samples of the 'Sad' class
df[df['class']=='Sad']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Sad,0.524292,0.810037,-1.279028,0.989107,0.565088,0.741143,-1.179697,0.988755,0.589510,...,-0.003031,0.0,0.615876,0.721075,0.021968,0.0,0.622517,0.713997,0.022469,0.0
1,Sad,0.523562,0.810187,-1.380470,0.990079,0.561935,0.740872,-1.278919,0.989783,0.584893,...,-0.003120,0.0,0.614124,0.720224,0.020959,0.0,0.620538,0.713706,0.021380,0.0
2,Sad,0.521723,0.809525,-1.400822,0.990983,0.558585,0.739532,-1.293301,0.990727,0.580836,...,-0.001872,0.0,0.610909,0.721197,0.022571,0.0,0.617341,0.714742,0.022983,0.0
3,Sad,0.520213,0.809108,-1.388353,0.991818,0.556583,0.738727,-1.276241,0.991590,0.578610,...,-0.002967,0.0,0.609726,0.720498,0.021407,0.0,0.616134,0.713853,0.021858,0.0
4,Sad,0.519008,0.808903,-1.382275,0.992485,0.555177,0.738464,-1.242853,0.992276,0.577124,...,-0.002414,0.0,0.605670,0.720482,0.021442,0.0,0.612132,0.713920,0.021803,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,Sad,0.550020,0.798606,-1.270250,0.997229,0.584543,0.735974,-1.160925,0.997550,0.605080,...,0.003651,0.0,0.625393,0.718925,0.026383,0.0,0.631905,0.709930,0.027340,0.0
229,Sad,0.549810,0.798601,-1.289255,0.997223,0.583556,0.735997,-1.182515,0.997543,0.604184,...,0.004348,0.0,0.629367,0.718818,0.028702,0.0,0.635676,0.709498,0.029846,0.0
230,Sad,0.554646,0.797065,-1.262392,0.997378,0.585263,0.734582,-1.151610,0.997673,0.605479,...,0.006230,0.0,0.640421,0.715205,0.032732,0.0,0.646381,0.706184,0.034013,0.0
231,Sad,0.563930,0.794357,-1.240011,0.997520,0.591189,0.732023,-1.126325,0.997798,0.611293,...,0.007486,0.0,0.644729,0.715257,0.034554,0.0,0.650972,0.706048,0.035943,0.0


In [79]:
X = df.drop('class', axis=1) # Store the Features
y = df['class'] # Store the Target value (i.e. Class Name)

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [81]:
y_test

228      Sad
69       Sad
170      Sad
40       Sad
92     Happy
       ...  
173      Sad
202      Sad
27       Sad
65       Sad
155      Sad
Name: class, Length: 86, dtype: object

## 3.2 Train Classification Model

In [82]:
# Build a pipeline object of different models to test
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [83]:
# Create a dictionary to store all the pipelines after they have been fitted
fit_models = {}

for algo, pipeline in pipelines.items():
    
    # [Note] Feature names have also been included in the scaled data so need to use .values. 
    ## Read more at https://stackoverflow.com/questions/69326639/sklearn-warning-valid-feature-names-in-version-1-0
    model = pipeline.fit(X_train.values, y_train.values) 
    fit_models[algo] = model

C:\Users\msyee\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [84]:
# Display all the fitted pipelines
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [85]:
# Predict target for test data using one of the fitted pipeline objects
fit_models['rc'].predict(X_test.values)

array(['Sad', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad',
       'Happy', 'Happy', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad',
       'Sad', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad',
       'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy',
       'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad',
       'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Happy', 'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad',
       'Sad'], dtype='<U5')

## 3.3 Evaluate and Export Model 

In [86]:
# Display the model test accuracy score for all the fitted models 
for algo, model in fit_models.items():
    yhat = model.predict(X_test.values)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 0.9767441860465116


In [87]:
# Generate predictions for test data using best performing pipeline object
fit_models['rf'].predict(X_test.values)

array(['Sad', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad',
       'Happy', 'Happy', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad',
       'Sad', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad',
       'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy',
       'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad',
       'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad',
       'Happy', 'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad',
       'Sad'], dtype=object)

In [88]:
# Display actual target values for test data
y_test

228      Sad
69       Sad
170      Sad
40       Sad
92     Happy
       ...  
173      Sad
202      Sad
27       Sad
65       Sad
155      Sad
Name: class, Length: 86, dtype: object

In [89]:
# Export the chosen pipeline to a .pkl file so that trained model run faster during for real-time predictions
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [90]:
# Load the pipeline
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [91]:
# Display the pipeline
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [92]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks - uncomment if you would like to see the landmarks during predictions
        '''mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )'''
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concatenate rows
            row = pose_row + face_row
            
#             # Append class name - uncomment if you would like to add more rows for model re-training in future 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        # press q key to terminate webcam capture mode
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Sad [0.41 0.59]
Sad [0.11 0.89]
Sad [0.09 0.91]
Sad [0.07 0.93]
Sad [0.06 0.94]
Sad [0.07 0.93]
Sad [0.07 0.93]
Sad [0.12 0.88]
Sad [0.19 0.81]
Sad [0.19 0.81]
Sad [0.27 0.73]
Sad [0.28 0.72]
Sad [0.3 0.7]
Sad [0.38 0.62]
Sad [0.42 0.58]
Sad [0.39 0.61]
Sad [0.37 0.63]
Sad [0.4 0.6]
Sad [0.38 0.62]
Sad [0.39 0.61]
Sad [0.42 0.58]
Sad [0.42 0.58]
Sad [0.36 0.64]
Sad [0.39 0.61]
Sad [0.49 0.51]
Sad [0.44 0.56]
Sad [0.48 0.52]
Sad [0.43 0.57]
Happy [0.57 0.43]
Happy [0.59 0.41]
Happy [0.57 0.43]
Happy [0.6 0.4]
Happy [0.56 0.44]
Happy [0.55 0.45]
Happy [0.55 0.45]
Sad [0.48 0.52]
Happy [0.53 0.47]
Happy [0.59 0.41]
Happy [0.5 0.5]
Sad [0.25 0.75]
Sad [0.18 0.82]
Sad [0.25 0.75]
Sad [0.35 0.65]
Happy [0.5 0.5]
Sad [0.47 0.53]
Happy [0.54 0.46]
Happy [0.56 0.44]
Happy [0.62 0.38]
Happy [0.63 0.37]
Happy [0.62 0.38]
Happy [0.62 0.38]
Happy [0.59 0.41]
Happy [0.56 0.44]
Happy [0.57 0.43]
Sad [0.44 0.56]
Sad [0.4 0.6]
Sad [0.44 0.56]
Sad [0.48 0.52]
Sad [0.48 0.52]
Sad [0.49 0.51]
Sad [0.44 0.

In [93]:
# [Optional] To display the X and Y coordinates of the Left Ear Landmark
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(453, 377)